# Python Notebook

In [ ]:
        // Check for image load completion
        var imagePromises = Array.from(images).map(function (img) {
            return new Promise(function (resolve, reject) {
                if (img.complete) {
                    resolve(); // If already loaded
                } else {
                    img.onload = resolve; // Resolve promise when image is loaded
                    img.onerror = reject; // Reject promise if image fails to load
                }
            });
        });
    
        // Wait for all images to load before generating the PDF
        Promise.all(imagePromises)
            .then(function () {
                console.log("All images loaded successfully.");
                downloadPDF();
            })
            .catch(function (error) {
                console.error("Image loading error:", error);
                downloadPDF(); // Proceed even if there's an error
            });
    };
    // uncomment until here if not needed

In [ ]:
def add_row(change):
    global gene_pair
    # Add a new row at the top with None values
    new_row = {col: None for col in gene_pair.columns}
    gene_pair = pd.DataFrame([new_row] + gene_pair.to_dict(orient="records"))
    update_table()

# Function to remove the last row of the dataframe
def remove_row(change):
    global gene_pair
    if len(gene_pair) > 0:
        gene_pair = gene_pair[:-1]  # Remove the last row
        update_table()

In [ ]:
gene_pair.columns

In [ ]:
duplicates = gene_pair00[gene_pair00["Human LR Pair"].duplicated()]
print(duplicates["Human LR Pair"])

In [2]:
## Function to create horizontal bar plots of each gene in Human Taxon --expression log(x+1) transformed with cell types as y-axis

import requests
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go

sys.path.append(os.path.abspath("src"))  # Add src directory to path
from createDataTable import gene_pair0

# Input file
input_file="data/connectome_j.tsv" #"data/connectome_j.tsv" # data/ExpressionGenes.txt
# Get all unique genes
ligand_list = gene_pair0["Ligand"].tolist()
receptor_list = gene_pair0["Receptor"].tolist()
unique_genes = list(set(ligand_list + receptor_list))  # Combine and remove duplicates

connectomeDB = pd.read_table(input_file, sep="\t")
# All Taxon for now
#connectomeDB = connectomeDB[connectomeDB["Taxon"]== "Human"]
if "Taxon" in connectomeDB.columns:
    connectomeDB = connectomeDB.drop(columns=["Localization", "Taxon"] + [col for col in connectomeDB.columns if col.startswith("F5_")])

In [3]:
column_sums = connectomeDB.iloc[:, 1:].sum()

In [5]:
connectomeDB.iloc[:, 1:].sum()

Adipocyte Breast           819735.859
Adipocyte Omental          883713.058
Adipocyte Perirenal        944323.351
Adipocyte Subcutaneous     805931.943
Alveolar Epithelial       1212346.313
                             ...     
Synoviocyte                769408.174
Tenocyte                   818224.667
Trabecular Meshwork       1048484.198
Tracheal Epithelial       1224157.145
Urothelial                 782699.893
Length: 144, dtype: float64

In [7]:
intersection = pd.Series(list(set(connectomeDB['ApprovedSymbol']).intersection(unique_genes)))
intersection

connectomeDB = connectomeDB[connectomeDB["ApprovedSymbol"].isin(intersection)]
connectomeDB
    

,ApprovedSymbol,Adipocyte Breast,Adipocyte Omental,Adipocyte Perirenal,Adipocyte Subcutaneous,Alveolar Epithelial,Amniotic Epithelial,Amniotic Membrane,Annulus Pulposus,Astrocyte Cerebellum,...,Smooth Muscle Subclavian Artery,Smooth Muscle Tracheal,Smooth Muscle Umbilical Artery,Smooth Muscle Umbilical Vein,Smooth Muscle Uterine,Synoviocyte,Tenocyte,Trabecular Meshwork,Tracheal Epithelial,Urothelial
2,A2M,90.272,121.423,50.596,63.397,0.000,0.000,0.819,0.000,1.364,...,0.146,37.598,2.324,0.143,3.193,2.208,5.266,0.000,0.223,0.000
16,AANAT,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
28,ABCA1,35.775,39.428,15.702,53.803,4.630,4.476,15.274,13.863,7.470,...,48.214,8.643,24.584,22.316,26.252,12.729,16.151,7.366,16.106,26.339
133,ACE,0.973,1.616,0.000,2.308,0.000,0.132,0.819,0.765,0.878,...,0.513,4.649,0.606,0.336,1.419,28.083,17.078,1.634,0.447,0.115
140,ACKR2,22.320,18.048,20.064,2.658,12.704,43.018,89.229,0.537,0.987,...,0.368,0.868,0.179,0.313,0.000,1.258,0.875,0.000,0.511,7.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16424,XCR1,0.000,0.000,0.000,0.000,0.000,0.000,0.656,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
16470,YBX1,58.803,54.388,58.447,73.160,76.354,80.877,43.623,85.979,113.238,...,67.607,104.832,91.272,102.638,51.795,57.435,74.201,106.607,67.368,69.804
16676,ZG16B,0.091,0.828,6.106,0.000,0.698,0.218,0.000,0.579,0.146,...,5.168,0.000,1.987,0.000,1.419,1.233,0.442,0.190,1.655,10.423
17213,ZNRF3,2.787,4.775,0.872,0.944,8.512,4.954,6.021,2.709,8.275,...,1.799,3.743,2.169,2.564,1.774,4.336,3.029,8.329,5.204,5.714


In [10]:
# log(x+1) transform
connectomeDB.iloc[:, 1:] = np.log1p(connectomeDB.iloc[:, 1:])
# Reshape 
connectomeDB_long = connectomeDB.melt(id_vars=["ApprovedSymbol"], 
                                      var_name="cellTypes", value_name="expr_val")
cellCat = pd.read_csv("data/cell_categories.csv")
connectomeDB_long = connectomeDB_long.merge(cellCat, how='left', left_on='cellTypes', right_on='cellType')
connectomeDB_long = connectomeDB_long.drop(columns=["cellType"])

intersection = pd.Series(list(set(connectomeDB_long['cellTypes']).intersection(set(cellCat['cellType']))))
intersection

diff_df = pd.Series(list(set(connectomeDB_long['cellTypes']).difference(set(cellCat['cellType']))))
diff_df

def plot_gene_expression(df):
    # Define the colors for each cell category
    colors = {
        "missing": "#B0B0B0",  # Neutral gray
        "other": "#D4A76A",  # Warm gold
        "mesenchymal": "#377EB8",  # Vibrant blue
        "epithelial": "#E41A1C",  # Bold red
        "hematopoietic": "#4DAF4A",  # Fresh green
        "endothelial": "#984EA3",  # Deep purple
        "nervous system": "#FF7F00",  # Bright orange
    }

    # Define sorting order for cell categories
    category_order = {cat: i for i, cat in enumerate(colors.keys())}

    for gene, sub_df in df.groupby("ApprovedSymbol"):
        # Sort by category first, then by expression value (highest first)
        sub_df = sub_df.copy()
        sub_df["category_order"] = sub_df["cellCategory"].map(category_order).fillna(len(category_order))
        sub_df = sub_df.sort_values(["category_order", "expr_val"], ascending=[True, False])

        num_bars = len(sub_df)

        # Plotly Figure setup
        fig = go.Figure()

        # Loop through each category and create a trace for it
        for category, color in colors.items():
            # Filter data for the current category
            category_data = sub_df[sub_df["cellCategory"] == category]

            # Add the trace for the current category
            fig.add_trace(go.Bar(
                y=category_data["cellTypes"],  # Categories for y-axis
                x=category_data["expr_val"],  # Expression values for x-axis
                orientation='h',  # Horizontal bars
                marker=dict(color=color),
                hovertemplate=
                    '<b>%{y}</b><br>' +  # Cell type (y-axis value)
                    'Expression Value: %{x}',  # Expression value (x-axis value)
                    #'Category: %{text}',  # Custom text (cell category)
                #text=category_data["cellCategory"],  # Pass the cell category as custom text
                name=category,  # Use the category name for the legend
                showlegend=True,  # Ensure the legend is shown for this trace
            ))

        # Update layout settings
        fig.update_layout(
            title="",
            xaxis_title="log(x+1) Expression value",
            yaxis_title="Cell Types",
            yaxis=dict(
                tickmode='array',
                tickvals=np.arange(num_bars),
                ticktext=sub_df["cellTypes"],
                tickangle=0,  # Avoid overlapping labels by setting the angle to 0
                tickfont=dict(size=6),  # Set font size for the labels
            ),
            showlegend=True,
            legend_title="Cell Category",
            legend=dict(
                orientation="v",  # Vertical legend
                yanchor="top",
                y=1,
                xanchor="",
                x=1.05,  # Position the legend outside of the plot area
                font=dict(size=10)
            ),
            margin=dict(t=50, b=50, l=150, r=50),
            height=min(1000, max(500, num_bars * 30)),  # Adjust plot height
            plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
            paper_bgcolor='rgba(0,0,0,0)',  # Transparent paper background
        )

        # Save to HTML file
        fig.write_html(f"data/gene_expr_plots/{gene}.html")


plot_gene_expression(connectomeDB_long)

In [1]:
connectomeDB_long

NameError: name 'connectomeDB_long' is not defined

## Testing Liana+

In [3]:
import liana as li
import omnipath as op
import decoupler as dc
import pandas as pd

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
import sys
import os
sys.path.append(os.path.abspath("src"))  # Add src directory to path
from createDataTable import gene_pair0

### Pathway Annotations

In [ ]:
# load PROGENy pathways, we use decoupler as a proxy as it formats the data in a more convenient way
progeny = dc.get_progeny(top=10000)
progeny

In [ ]:
lr_pairs = gene_pair0[["Ligand", "Receptor"]]
lr_pairs.columns = lr_pairs.columns.str.lower()

In [ ]:
lr_pairs

In [ ]:
# generate ligand-receptor geneset
lr_progeny = li.rs.generate_lr_geneset(lr_pairs, progeny, lr_sep="^")

In [ ]:
lr_progeny

In [ ]:
# some of the pairs are missing
len(lr_progeny["interaction"].unique())

In [ ]:
output_file="data/pathway_annotations_per_pair.csv"
lr_progeny.to_csv(output_file, index=False)

In [ ]:
whichDB= 'DisGeNet'
# A database of expression profiles related to human diseases, including cancer
diseases = op.requests.Annotations.get(
    resources = [whichDB]
    )

In [ ]:
diseases

In [ ]:
diseases.to_csv("data/" + whichDB + ".csv")

### Disease Annotations

In [ ]:
# DisGeNet
diseases = op.requests.Annotations.get(
    resources = ['DisGeNet']
    )

In [ ]:
diseases = diseases[['genesymbol', 'label', 'value']]
diseases = diseases.pivot_table(index='genesymbol',
                                columns='label', values='value',
                                aggfunc=lambda x: '; '.join(x)).reset_index()
diseases = diseases[['genesymbol', 'disease']]
diseases['disease'] = diseases['disease'].str.split('; ')
diseases = diseases.explode('disease')
lr_diseases = li.rs.generate_lr_geneset(lr_pairs, diseases, source='disease', target='genesymbol', weight=None, lr_sep="^")
lr_diseases.sort_values("interaction")

In [ ]:
# some of the pairs are missing
len(lr_diseases["interaction"].unique())

In [ ]:
output_file="data/disease_annotations_per_pair.csv"
lr_diseases.to_csv(output_file, index=False)

In [ ]:
op.requests.Annotations.resources()

### Get FASTA sequences for each gene

In [ ]:
import requests
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath("src"))  # Add src directory to path
from createDataTable import gene_pair0

# Get all unique genes
ligand_list = gene_pair0["Ligand"].tolist()
receptor_list = gene_pair0["Receptor"].tolist()
unique_genes = list(set(ligand_list + receptor_list))  # Combine and remove duplicates

In [ ]:
import re
import pandas as pd
import gzip

# Input FASTA file
fasta_file = "data/uniprotkb_proteome_UP000005640_AND_revi_2025_03_27.fasta.gz"

# Regex pattern to extract details from header
header_pattern = re.compile(r"^>sp\|(?P<uniprot_id>[A-Z0-9]+(?:-\d+)?)\|(?P<protein_name>.+?) OS=Homo sapiens OX=9606 GN=(?P<gene_name>[A-Za-z0-9-]+)")

# Store extracted data
records = []

# Read and parse the file
with gzip.open(fasta_file, "rt") as f:
    header = None
    sequence = []
    
    for line in f:
        line = line.strip()
        
        if line.startswith(">"):
            # Store previous sequence if exists
            if header and sequence:
                isoform_type = "Canonical" if "-" not in header["uniprot_id"] else "Alternative Isoform"
                records.append([header["uniprot_id"], header["gene_name"], header["protein_name"], isoform_type, "".join(sequence)])
            
            # Match new header
            match = header_pattern.match(line)
            if match:
                header = match.groupdict()
                sequence = []
            else:
                header = None
        
        elif header:
            sequence.append(line)

    # Add the last record
    if header and sequence:
        isoform_type = "Canonical" if "-" not in header["uniprot_id"] else "Alternative Isoform"
        records.append([header["uniprot_id"], header["gene_name"], header["protein_name"], isoform_type, "".join(sequence)])

# Convert to pandas DataFrame
df = pd.DataFrame(records, columns=["UniProt ID", "Gene Symbol", "Protein Name", "Isoform Type", "FASTA Sequence"])

# Save as TSV
df.to_csv("data/human_uniprot_isoforms.tsv", sep="\t", index=False)

print(f"✅ Extracted {len(df)} Homo sapiens protein sequences and saved to 'human_uniprot_isoforms.tsv'.")


In [ ]:
len(df["Gene Symbol"].unique())

In [ ]:
df= pd.read_table("data/human_uniprot_isoforms.tsv", sep="\t")

In [ ]:
df.columns

In [ ]:
df = df[['UniProt ID', 'Gene Symbol', 'Isoform Type', 'FASTA Sequence']]

In [ ]:
df

In [ ]:
lim_df = gene_pair0[["Human LR Pair", "Ligand", "Receptor"]]

In [ ]:
lim_df

In [ ]:
lim_df = lim_df.merge(df, how='left', left_on='Ligand', right_on='Gene Symbol')
lim_df = lim_df.drop(columns=["Gene Symbol"])
lim_df = lim_df.rename(columns={"FASTA Sequence": "Ligand Sequence",
                                "UniProt ID": "Ligand Isoform Uniprot ID",
                                "Isoform Type": "Ligand Isoform Type"})
lim_df

In [ ]:
lim_df = lim_df.merge(df, how='left', left_on='Receptor', right_on='Gene Symbol')
lim_df = lim_df.drop(columns=["Gene Symbol"])
lim_df = lim_df.rename(columns={"FASTA Sequence": "Receptor Sequence",
                                "UniProt ID": "Receptor Isoform Uniprot ID",
                                "Isoform Type": "Receptor Isoform Type"})
lim_df

In [ ]:
lim_df.to_csv("data/LRpair_uniprot_sequences.tsv", sep="\t", index=False)

In [ ]:
import gzip
import re
import pandas as pd

# Step 1: Extract Gene Symbol Mapping from GTF
gtf_file = "data/gencode.v47.annotation.gtf.gz"
gene_map = {}

# Read GTF file and extract gene_id -> gene_name mapping
with gzip.open(gtf_file, "rt") as f:
    for line in f:
        if line.startswith("#"):  # Skip comments
            continue
        
        fields = line.strip().split("\t")
        if fields[2] == "gene":  # Only extract gene entries
            info = {key.strip(): value.strip('"') for key, value in re.findall(r'(\S+) "([^"]+)"', fields[8])}
            if "gene_id" in info and "gene_name" in info:
                gene_map[info["gene_id"]] = info["gene_name"]

print(f"✅ Extracted {len(gene_map)} gene mappings from GTF.")

In [ ]:
# Step 2: Parse GENCODE Protein FASTA and Add Gene Symbols
fasta_file = "data/gencode.v47.pc_translations.fa.gz"

# Store extracted data
records = []

# Open the GENCODE FASTA file and parse sequences
with gzip.open(fasta_file, "rt") as f:
    header = None
    sequence = []
    
    for line in f:
        line = line.strip()
        
        if line.startswith(">"):
            # Store previous sequence if exists
            if header and sequence:
                # Extract the Gene Symbol using the Gene ID
                gene_symbol = gene_map.get(header["gene_id"], "Unknown")
                isoform_type = "Canonical" if "-1" in header["protein_id"] else "Alternative Isoform"
                
                # Append the parsed data to records
                records.append([header["protein_id"], header["transcript_id"], header["gene_id"], gene_symbol, isoform_type, "".join(sequence)])
            
            # Split header by '|' and extract necessary fields
            fields = line[1:].split("|")  # Skip the '>' symbol and split by '|'
            if len(fields) >= 6:
                header = {
                    "protein_id": fields[0], 
                    "transcript_id": fields[1], 
                    "gene_id": fields[2]  
                }
                sequence = []
            else:
                header = None
        
        elif header:
            sequence.append(line)

    # Add the last record if needed
    if header and sequence:
        gene_symbol = gene_map.get(header["gene_id"], "Unknown")
        isoform_type = "Canonical" if "-1" in header["protein_id"] else "Alternative Isoform"
        records.append([header["protein_id"], header["transcript_id"], header["gene_id"], gene_symbol, isoform_type, "".join(sequence)])

# Step 3: Convert to pandas DataFrame and Save to TSV
df = pd.DataFrame(records, columns=["Ensembl Protein ID", "Ensembl Transcript ID", "Ensembl Gene ID", "Gene Symbol", "Isoform Type", "FASTA Sequence"])

# Save to TSV
df.to_csv("data/gencode_protein_isoforms_with_symbols.tsv", sep="\t", index=False)

# Print completion message
print(f"✅ Extracted {len(df)} protein sequences with Gene Symbols and saved to 'gencode_protein_isoforms_with_symbols.tsv'.")

In [ ]:
lim_df = gene_pair0[["Human LR Pair", "Ligand", "Receptor"]]
lim_df = lim_df.merge(df, how='left', left_on='Ligand', right_on='Gene Symbol')

In [ ]:
lim_df

In [ ]:
lim_df = lim_df.drop(columns=["Gene Symbol"])
lim_df = lim_df.rename(columns={"FASTA Sequence": "Ligand Sequence",
                                "Ensembl Protein ID": "Ligand Ensembl Protein ID",
                                "Ensembl Transcript ID": "Ligand Ensembl Transcript ID",
                                "Ensembl Gene ID": "Ligand Ensembl Gene ID",
                                "Isoform Type": "Ligand Isoform Type"})

In [ ]:
lim_df

In [ ]:
lim_df = lim_df.merge(df, how='left', left_on='Receptor', right_on='Gene Symbol')
lim_df = lim_df.drop(columns=["Gene Symbol"])
lim_df = lim_df.rename(columns={"FASTA Sequence": "Receptor Sequence",
                                "Ensembl Protein ID": "Receptor Ensembl Protein ID",
                                "Ensembl Transcript ID": "Receptor Ensembl Transcript ID",
                                "Ensembl Gene ID": "Receptor Ensembl Gene ID",
                                "Isoform Type": "Receptor Isoform Type"})

In [ ]:
lim_df

In [ ]:
lim_df.to_csv("data/LRpair_gencode_sequences.tsv", sep="\t", index=False)

####################################################################

In [54]:
## Function to scrape data from Pubmed for Title, Abstract, Journal, and Year
### IMPORTANT: TURN OFF VPN and make sure you have the data directory (from Sakura)

import sys
import requests
import pandas as pd
import time
import os
import xml.etree.ElementTree as ET

sys.path.append(os.path.abspath("src"))  
import fetchGSheet

# Read the API key from a file
with open("data/ncbi_api_key.txt", "r") as file:
    ncbi_api_key = file.read().strip()

# File to save the results
output_file = "data/pubmed_results.csv"

# Example of fetching HGNC gene symbols (you should have the `fetchGSheet.pop_up_info` dataframe ready)
def extract_hgnc_symbols(fetchGSheet):
    # Concatenate Approved, Alias, and Previous symbols, then extract unique symbols
    hgnc_symbols = pd.concat([
        fetchGSheet['Approved symbol'],
        fetchGSheet['Alias symbol'],
        fetchGSheet['Previous symbol']
    ], axis=0).dropna().str.upper().unique()  # Remove NaNs and make uppercase for matching
     # Remove any empty strings from the list
    hgnc_symbols = [symbol for symbol in hgnc_symbols if symbol != ""]
    return set(hgnc_symbols)  # Return as a set for fast lookup
    
hgnc_symbols = extract_hgnc_symbols(fetchGSheet.pop_up_info)

In [56]:
len(hgnc_symbols)

100941

In [ ]:
# Official species names and their corresponding terms (scientific names)
# Load your list of PMIDs
pmid_list = source
species_dict = {
    "human": "Homo sapiens",
    "mouse": "Mus musculus",
    "rat": "Rattus norvegicus",
    "rabbit": "Oryctolagus cuniculus",
    "monkey": "Macaca spp.",
    "dog": "Canis lupus familiaris",
    "pig": "Sus scrofa",
    "zebra fish": "Danio rerio",
    "chicken": "Gallus gallus",
    "horse": "Equus ferus caballus",
    "cat": "Felis catus",
    "sheep": "Ovis aries",
    "cow": "Bos taurus",
    "fruit fly": "Drosophila melanogaster",
    "c. elegans": "Caenorhabditis elegans",
}

def fetch_pubmed_data(pmid_list, hgnc_symbols):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    results = []

    # Load existing data if output file exists
    if os.path.exists(output_file):
        existing_data = pd.read_csv(output_file)
    else:
        existing_data = pd.DataFrame(columns=["PMID", "Title", "Abstract", "Journal", "Year", "Species"])

    # Split PMIDs into batches
    batch_size = 50
    pmid_batches = [pmid_list[i:i + batch_size] for i in range(0, len(pmid_list), batch_size)]

    # Iterate over the batches
    for batch in pmid_batches:
        params = {
            "db": "pubmed",
            "id": ",".join(batch),  # Join PMIDs as comma-separated
            "retmode": "xml",
            "api_key": ncbi_api_key
        }

        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()

            # Parse the XML response
            root = ET.fromstring(response.text)
            for article in root.findall(".//PubmedArticle"):
                # Extract Title and Abstract
                title = article.findtext(".//ArticleTitle", default="N/A")
                abstract = article.findtext(".//AbstractText", default="No abstract available")

                # Extract Journal Title
                journal_tag = article.find(".//Journal/Title")
                journal = journal_tag.text.strip() if journal_tag is not None and journal_tag.text else "N/A"

                # Extract Publication Year
                pub_date = article.find(".//PubDate")
                if pub_date is not None:
                    year_tag = pub_date.find("Year")
                    year = year_tag.text if year_tag is not None else "N/A"

                    # Fallback to MedlineDate if Year is missing
                    if year == "N/A":
                        medline_date_tag = pub_date.find("MedlineDate")
                        year = medline_date_tag.text.split()[0] if medline_date_tag is not None else "N/A"
                else:
                    year = "N/A"  # PubDate is completely missing

                # Initialize species as N/A
                species = "N/A"

                # Check if the word "patient" is detected in title or abstract (assume human)
                if "patient" in title.lower() or "patient" in abstract.lower():
                    species = "Homo sapiens"
                elif "human" in title.lower() or "human" in abstract.lower():
                    species = "Homo sapiens"
                else:
                    # Look for HGNC gene symbols in title or abstract (assume human if found)
                    for gene in hgnc_symbols:
                        if gene in title or gene in abstract:
                            species = "Homo sapiens"
                            break
                    else:
                        # Look for MeSH terms related to species
                        for mesh_heading in article.findall(".//MeshHeadingList/MeshHeading"):
                            descriptor_name = mesh_heading.findtext("DescriptorName")
                            if descriptor_name:
                                # Match official species names using the species_dict
                                for species_term, scientific_name in species_dict.items():
                                    if species_term in descriptor_name.lower():
                                        species = scientific_name
                                        break  # Stop after finding the first match

                # Append the result
                results.append({
                    "PMID": article.findtext(".//MedlineCitation/PMID"),
                    "Title": title,
                    "Abstract": abstract,
                    "Journal": journal,
                    "Year": year,
                    "Species": species
                })

        except Exception as e:
            print(f"Error fetching batch {batch}: {e}")
            # Optionally save the response for debugging
            with open(f"error_batch_{batch[0]}_{batch[-1]}.xml", "w") as f:
                f.write(response.text)

        # Rate limiting to avoid API overload
        time.sleep(1)  # Increase delay for better API compliance

    # Save results
    new_data = pd.DataFrame(results)
    if not new_data.empty:
        # Merge existing and new data, updating missing values
        updated_data = pd.concat([existing_data, new_data])

        # Ensure all PMIDs are strings
        updated_data["PMID"] = updated_data["PMID"].astype(str)

        # Drop rows with missing PMIDs
        updated_data = updated_data.dropna(subset=["PMID"])

        # Ensure rows are ordered and remove duplicates
        updated_data = (
            updated_data.sort_values(by="PMID")  # Ensure rows are ordered
            .drop_duplicates(subset="PMID", keep="last")  # Keep the latest data
        )
        updated_data["Journal"] = updated_data["Journal"].str.split(" (", n=1, expand=False, regex=False).str[0]
        updated_data.to_csv(output_file, index=False)
    else:
        print("No new data fetched.")

    return results

# Fetch PubMed data with your list of PMIDs, output file path, and NCBI API key
fetch_pubmed_data(pmid_list, hgnc_symbols)